In [118]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import numpy as np
from collections import Counter
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from imblearn.under_sampling import RandomUnderSampler
import spacy
import re
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight

In [150]:
# Load the TSV file
column_names = ["pmid", "entity1_type", "entity2_type", "entity1_id", "entity2_id",
                "random_bool", "random_integer", "text", "correlation", "novel"]

df = pd.read_csv("processed2/train.tsv", sep="\t", header=None, names=column_names)
test = pd.read_csv("processed2/test.tsv", sep="\t", header=None, names=column_names)

# Display first few rows to inspect the structure
print(df.head())

       pmid                entity1_type       entity2_type entity1_id  \
0  10491763           GeneOrGeneProduct     ChemicalEntity       3175   
1  10491763           GeneOrGeneProduct      OrganismTaxon       6927   
2  10491763  DiseaseOrPhenotypicFeature      OrganismTaxon    D003924   
3  10491763           GeneOrGeneProduct  GeneOrGeneProduct       3175   
4  10491763           GeneOrGeneProduct  GeneOrGeneProduct       3172   

  entity2_id  random_bool  random_integer  \
0    D005947         True               0   
1       9606         True               0   
2       9606         True               0   
3       6927        False               1   
4       6927         True               0   

                                                text correlation novel  
0  @GeneOrGeneProductSrc$ Hepatocyte nuclear fact...         NaN   NaN  
1  Hepatocyte nuclear factor-6 : associations bet...         NaN   NaN  
2  Hepatocyte nuclear factor-6 : associations bet...         NaN   NaN 

In [151]:
df.fillna({"correlation": "None"}, inplace=True)
df.fillna({"novel": "No"}, inplace=True)

test.fillna({"correlation": "None"}, inplace=True)
test.fillna({"novel": "No"}, inplace=True)


## Baseline 1

In [140]:
correlation_row = df['correlation']
# Find the most common value in the row
most_common_value = correlation_row.mode().iloc[0]
print(most_common_value)

None


In [45]:
# baseline 1 -- always predict the most common value
correct = 0

true_labels = test['correlation']
for i in range(len(true_labels)):
    if true_labels[i] == most_common_value:
        correct += 1

predicted_labels = [most_common_value] * len(true_labels)

print("Baseline 1 accuracy: ", accuracy_score(true_labels, predicted_labels))
print("Baseline precision: ", precision_score(true_labels, predicted_labels, average='macro'))
print("Baseline recall: ", recall_score(true_labels, predicted_labels, average='macro'))
# print("Baseline recall: ", recall_score(true_labels, predicted_labels))
print("Baseline F1: ", f1_score(true_labels, predicted_labels, average='macro'))
print("Baseline classification report: ")
print(classification_report(true_labels, predicted_labels, digits=4))

Baseline 1 accuracy:  0.8841864170483967
Baseline precision:  0.09824293522759964
Baseline recall:  0.1111111111111111
Baseline F1:  0.10428154490307653
Baseline classification report: 
                      precision    recall  f1-score   support

         Association     0.0000    0.0000    0.0000       635
                Bind     0.0000    0.0000    0.0000         9
          Comparison     0.0000    0.0000    0.0000         6
          Conversion     0.0000    0.0000    0.0000         1
         Cotreatment     0.0000    0.0000    0.0000        14
    Drug_Interaction     0.0000    0.0000    0.0000         2
Negative_Correlation     0.0000    0.0000    0.0000       171
                None     0.8842    1.0000    0.9385      8879
Positive_Correlation     0.0000    0.0000    0.0000       325

            accuracy                         0.8842     10042
           macro avg     0.0982    0.1111    0.1043     10042
        weighted avg     0.7818    0.8842    0.8298     10042



/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

## Baseline 2

In [141]:
def get_features():
    vectorizer = TfidfVectorizer()

    vec_text = vectorizer.fit_transform(df["text"])
    vec_text_test = vectorizer.transform(test["text"])

    entity_mapping = {entity: idx for idx, entity in enumerate(set(df["entity1_type"]))}
    entity1_type = np.array([entity_mapping[entity] for entity in df["entity1_type"]])
    entity2_type = np.array([entity_mapping[entity] for entity in df["entity2_type"]])

    features = np.hstack([vec_text.toarray(), entity1_type.reshape(-1, 1), entity2_type.reshape(-1, 1)])

    entity1_type_test = np.array([entity_mapping[entity] for entity in test["entity1_type"]])
    entity2_type_test = np.array([entity_mapping[entity] for entity in test["entity2_type"]])

    features = np.hstack([vec_text.toarray(), entity1_type.reshape(-1, 1), entity2_type.reshape(-1, 1)])
    features_test = np.hstack([vec_text_test.toarray(), entity1_type_test.reshape(-1, 1), entity2_type_test.reshape(-1, 1)])

    # generate labels
    label_mapping = {label: idx for idx, label in enumerate(set(df["correlation"]))}
    labels = np.array([label_mapping[label] for label in df["correlation"]])
    # print(labels)
    y_test = np.array([label_mapping[label] for label in test["correlation"]])

    return features, labels, features_test, y_test, label_mapping

In [ ]:
features, labels, features_test, y_test, label_mapping = get_features()

clf = LogisticRegression(max_iter=1000)
clf.fit(features, labels)

# Evaluate
y_pred = clf.predict(features_test)

print(classification_report(y_test, y_pred, target_names=label_mapping.keys(), digits=4))

                      precision    recall  f1-score   support

Negative_Correlation     0.2718    0.3099    0.2896       171
         Association     0.2938    0.2378    0.2628       635
    Drug_Interaction     0.0000    0.0000    0.0000         2
         Cotreatment     0.0000    0.0000    0.0000        14
                None     0.9214    0.9616    0.9411      8879
          Conversion     0.0000    0.0000    0.0000         1
Positive_Correlation     0.3582    0.0738    0.1224       325
          Comparison     0.0000    0.0000    0.0000         6
                Bind     0.0000    0.0000    0.0000         9

            accuracy                         0.8729     10042
           macro avg     0.2050    0.1759    0.1796     10042
        weighted avg     0.8495    0.8729    0.8576     10042



/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

## Normalizing tagged entities

In [142]:
nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("scispacy_linker", config={"linker_name": "umls"})

/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [143]:
# Replace all entries in the tags with the same entity name
# have the two entities as a separate features
# denote distance between the entities (average?) as a feature

def get_canonical_name(entity_text):
    """Returns the canonical UMLS name for an entity, or None if no match found."""
    doc = nlp(entity_text)
    for ent in doc.ents:
        umls_concepts = ent._.kb_ents  # Get UMLS linking information
        if umls_concepts:
            umls_id = umls_concepts[0][0]  # Top UMLS Concept ID
            return nlp.get_pipe("scispacy_linker").kb.cui_to_entity[umls_id].canonical_name
    return None


def normalize_tagged_entities(tag_pattern, text):
    """Replaces words within any @somethingSrc$ or @somethingTgt$ tag with their canonical UMLS names."""
    tag_replacements = {}  # Stores {original_word: canonical_word}
    canonical_name = None
    # Extract and process all matching tags
    matches = re.findall(tag_pattern, text)
    for full_match in matches:
        tag_type, _, entity_text = full_match
        if canonical_name:
            tag_replacements[entity_text] = canonical_name
        else:
            canonical_name = get_canonical_name(entity_text)
            if not canonical_name:
                assert(entity_text)
                canonical_name = entity_text
                tag_replacements[entity_text] = canonical_name


    def replace_match(match):
        tag_type, _, entity_text = match.groups()
        replacement = tag_replacements.get(entity_text, entity_text)
        return f"@{tag_type}$ {replacement} @/{tag_type}$"

    text = re.sub(tag_pattern, replace_match, text)
    
    return text, canonical_name

In [144]:
src_pattern = r"@(\w+(Src))\$\s*(.*?)\s*@/\1\$"
tgt_pattern = r"@(\w+(Tgt))\$\s*(.*?)\s*@/\1\$"

df[["text", "entity_1"]] = df["text"].apply(lambda x: pd.Series(normalize_tagged_entities(src_pattern, x)))
df[["text", "entity_2"]] = df["text"].apply(lambda x: pd.Series(normalize_tagged_entities(tgt_pattern, x)))

In [145]:
test[["text", "entity_1"]] = test["text"].apply(lambda x: pd.Series(normalize_tagged_entities(src_pattern, x)))
test[["text", "entity_2"]] = test["text"].apply(lambda x: pd.Series(normalize_tagged_entities(tgt_pattern, x)))

In [ ]:
columns_to_display = ['text', 'entity_1', 'entity_2', 'correlation']
# Display the first few rows with specific columns
print(df[columns_to_display].head())

# Display the first few rows with specific columns
print(test[columns_to_display].head())

In [146]:
features, labels, features_test, y_test, label_mapping = get_features()

clf_keywords = LogisticRegression(max_iter=1000)
clf_keywords.fit(features, labels)

# Evaluate
y_pred = clf_keywords.predict(features_test)

print(classification_report(y_test, y_pred, target_names=label_mapping.keys(), digits=4))

                      precision    recall  f1-score   support

Negative_Correlation     0.2045    0.1579    0.1782       171
         Association     0.3018    0.1858    0.2300       635
    Drug_Interaction     0.0000    0.0000    0.0000         2
         Cotreatment     0.0000    0.0000    0.0000        14
                None     0.9127    0.9675    0.9393      8879
          Conversion     0.0000    0.0000    0.0000         1
Positive_Correlation     0.3458    0.1138    0.1713       325
          Comparison     0.0000    0.0000    0.0000         6
                Bind     0.0000    0.0000    0.0000         9

            accuracy                         0.8735     10042
           macro avg     0.1961    0.1583    0.1688     10042
        weighted avg     0.8407    0.8735    0.8536     10042



/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

## Adding distance between tagged entities as a feature

In [152]:
def extract_entities(text):
    """Extract entities based on the provided tags."""
    # Pattern to match the tags and extract the entity text
    src_pattern = r"@[\w]+Src\$"
    tgt_pattern = r"@[\w]+Tgt\$"
    
    # Find all matches for both source and target tags
    entities_src = re.findall(src_pattern, text)
    entities_tgt = re.findall(tgt_pattern, text)
    # print(entities_src)
    # print(entities_tgt)
    return entities_src, entities_tgt

def calculate_distances(df):
    distances = []
    
    for text in df["text"]:
        # Extract entities based on tags
        entities_src, entities_tgt = extract_entities(text)
        
        # Tokenize the text
        tokens = text.split()
        
        # Find the positions of each entity in the tokenized text
        src_positions = [i for i, token in enumerate(tokens) if token in entities_src]
        tgt_positions = [i for i, token in enumerate(tokens) if token in entities_tgt]
        
        if src_positions and tgt_positions:
            min_distance = float('inf')
            # Calculate distances between all occurrences
            for src_pos in src_positions:
                for tgt_pos in tgt_positions:
                    distance = abs(src_pos - tgt_pos)
                    if distance < min_distance:
                        min_distance = distance
            distances.append(min_distance)
        else:
            distances.append(-1)  # Placeholder for no occurrence
    
    return distances

# calculate_distances(df, "extract_entities(text)@GeneOrGeneProductSrc$ Hepatocyte nuclear factor-6 @/GeneOrGeneProductSrc$ : associations between genetic variability and type II diabetes and between genetic variability and estimates of insulin secretion . The transcription factor @GeneOrGeneProductSrc$ hepatocyte nuclear factor (HNF)-6 @/GeneOrGeneProductSrc$ is an upstream regulator of several genes involved in the pathogenesis of maturity-onset diabetes of the young . We therefore tested the hypothesis that variability in the @GeneOrGeneProductSrc$ HNF-6 @/GeneOrGeneProductSrc$ gene is associated with subsets of Type II ( non-insulin-dependent ) diabetes mellitus and estimates of insulin secretion in @ChemicalEntityTgt$ glucose @/ChemicalEntityTgt$ tolerant subjects . We cloned the coding region as well as the intron-exon boundaries of the @GeneOrGeneProductSrc$ HNF-6 @/GeneOrGeneProductSrc$ gene . W e then examined them on genomic DNA in six MODY probands without mutations in the MODY1 , MODY3 and MODY4 genes and in 54 patients with late-onset Type II diabetes by combined single strand conformational polymorphism-heteroduplex analysis followed by direct sequencing of identified variants . An identified missense variant was examined in association studies and genotype-phenotype studies . We identified two silent and one missense ( Pro75 Ala ) variant . I n an association study the allelic frequency of the Pro75Ala polymorphism was 3.2 % ( 95 % confidence interval , 1.9 - 4.5 ) in 330 patients with Type II diabetes mellitus compared with 4.2 % ( 2.4 - 6.0 ) in 238 age-matched @ChemicalEntityTgt$ glucose @/ChemicalEntityTgt$ tolerant control subjects . Moreover , in studies of 238 middle-aged @ChemicalEntityTgt$ glucose @/ChemicalEntityTgt$ tolerant subjects , of 226 @ChemicalEntityTgt$ glucose @/ChemicalEntityTgt$ tolerant offspring of Type II diabetic patients and of 367 young healthy subjects , the carriers of the polymorphism did not differ from non-carriers in @ChemicalEntityTgt$ glucose @/ChemicalEntityTgt$ induced serum insulin or C-peptide responses . Mutations in the coding region of the @GeneOrGeneProductSrc$ HNF-6 @/GeneOrGeneProductSrc$ gene are not associated with Type II diabetes or with changes in insulin responses to @ChemicalEntityTgt$ glucose @/ChemicalEntityTgt$ among the Caucasians examined")
df["distance"] = calculate_distances(df)
test["distance"] = calculate_distances(test)

In [ ]:
print(df[["distance"]].head())
print(test[["distance"]].head())

In [153]:
vectorizer = TfidfVectorizer()

vec_text = vectorizer.fit_transform(df["text"])
vec_text_test = vectorizer.transform(test["text"])

entity_mapping = {entity: idx for idx, entity in enumerate(set(df["entity1_type"]))}
entity1_type = np.array([entity_mapping[entity] for entity in df["entity1_type"]])
entity2_type = np.array([entity_mapping[entity] for entity in df["entity2_type"]])

entity1_type_test = np.array([entity_mapping[entity] for entity in test["entity1_type"]])
entity2_type_test = np.array([entity_mapping[entity] for entity in test["entity2_type"]])

features = np.hstack([vec_text.toarray(), entity1_type.reshape(-1, 1), entity2_type.reshape(-1, 1), df["distance"].values.reshape(-1, 1)])
features_test = np.hstack([vec_text_test.toarray(), entity1_type_test.reshape(-1, 1), entity2_type_test.reshape(-1, 1), test["distance"].values.reshape(-1, 1)])

# generate labels
label_mapping = {label: idx for idx, label in enumerate(set(df["correlation"]))}
labels = np.array([label_mapping[label] for label in df["correlation"]])
y_test = np.array([label_mapping[label] for label in test["correlation"]])

In [154]:
# features, labels, features_test, y_test, label_mapping = get_features()
clf_distance = LogisticRegression(max_iter=1000)
clf_distance.fit(features, labels)

# Evaluate
y_pred = clf_distance.predict(features_test)

print(classification_report(y_test, y_pred, target_names=label_mapping.keys(), digits=4))

/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                      precision    recall  f1-score   support

Negative_Correlation     0.2265    0.2398    0.2330       171
         Association     0.3165    0.2268    0.2642       635
    Drug_Interaction     0.0000    0.0000    0.0000         2
         Cotreatment     0.0000    0.0000    0.0000        14
                None     0.9194    0.9640    0.9412      8879
          Conversion     0.0000    0.0000    0.0000         1
Positive_Correlation     0.3958    0.1169    0.1805       325
          Comparison     0.0000    0.0000    0.0000         6
                Bind     0.0000    0.0000    0.0000         9

            accuracy                         0.8745     10042
           macro avg     0.2065    0.1719    0.1799     10042
        weighted avg     0.8496    0.8745    0.8587     10042



/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

## Oversampling from the minority classes

In [133]:
print(label_mapping)
minority_classes = ['Drug_Interaction', 'Cotreatment', 'Conversion', 'Comparison', 'Bind']
sampling_strategy = {label_mapping[label]: 300 for label in minority_classes}
random_oversampler = RandomOverSampler(random_state=42, sampling_strategy=sampling_strategy)
X_resampled, y_resampled = random_oversampler.fit_resample(features, labels)

{'Negative_Correlation': 0, 'Association': 1, 'Drug_Interaction': 2, 'Cotreatment': 3, 'None': 4, 'Conversion': 5, 'Positive_Correlation': 6, 'Comparison': 7, 'Bind': 8}


In [134]:
print("Original training set shape:", np.bincount(labels))
print("Undersampled dataset shape:", np.bincount(y_resampled))

Original training set shape: [  763  2191    11    31 26655     3  1088    28    60]
Undersampled dataset shape: [  763  2191   300   300 26655   300  1088   300   300]


In [135]:
clf2 = LogisticRegression(max_iter=1000)
clf2.fit(X_resampled, y_resampled)

/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [136]:
# Evaluate
y_pred = clf2.predict(features_test)
print(classification_report(y_test, y_pred, target_names=label_mapping.keys(), digits=4))

                      precision    recall  f1-score   support

Negative_Correlation     0.2185    0.1520    0.1793       171
         Association     0.3097    0.1858    0.2323       635
    Drug_Interaction     0.2500    1.0000    0.4000         2
         Cotreatment     0.3333    0.1429    0.2000        14
                None     0.9140    0.9688    0.9406      8879
          Conversion     0.0000    0.0000    0.0000         1
Positive_Correlation     0.4124    0.1231    0.1896       325
          Comparison     0.2667    0.6667    0.3810         6
                Bind     0.0000    0.0000    0.0000         9

            accuracy                         0.8757     10042
           macro avg     0.3005    0.3599    0.2803     10042
        weighted avg     0.8455    0.8757    0.8561     10042



/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [137]:
print(label_mapping)

{'Negative_Correlation': 0, 'Association': 1, 'Drug_Interaction': 2, 'Cotreatment': 3, 'None': 4, 'Conversion': 5, 'Positive_Correlation': 6, 'Comparison': 7, 'Bind': 8}


In [155]:
clf_balanced = LogisticRegression(class_weight="balanced", max_iter=1000)
clf_balanced.fit(features, labels)

# Evaluate
y_pred = clf_balanced.predict(features_test)

print(classification_report(y_test, y_pred, target_names=label_mapping.keys(), digits=4))

/opt/homebrew/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                      precision    recall  f1-score   support

Negative_Correlation     0.1210    0.3567    0.1807       171
         Association     0.1917    0.4898    0.2756       635
    Drug_Interaction     0.1818    1.0000    0.3077         2
         Cotreatment     0.1154    0.4286    0.1818        14
                None     0.9644    0.7817    0.8635      8879
          Conversion     0.0000    0.0000    0.0000         1
Positive_Correlation     0.1429    0.2646    0.1855       325
          Comparison     0.2381    0.8333    0.3704         6
                Bind     0.0000    0.0000    0.0000         9

            accuracy                         0.7381     10042
           macro avg     0.2173    0.4616    0.2628     10042
        weighted avg     0.8719    0.7381    0.7906     10042

